In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from scipy.stats import iqr
import os

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors
%matplotlib inline
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import geopandas
import geopandas as gpd
from geopandas.tools import sjoin

from shapely.geometry import Point

import datetime

from tqdm import tqdm
import networkx as nx

from HydroGraph_functions import *

In [5]:
#TODO: Refactor this and put the correct data in one place

WIRivers = pd.read_pickle("WIRivers.df")
WILakes  = pd.read_pickle("WILakes.df")
WIhuc8   = pd.read_pickle("WIHUC8.df")
WIhuc12  = pd.read_pickle("WIHUC12.df")
WIhuc10  = pd.read_pickle("WIHUC10.df")
WI = gpd.GeoDataFrame.from_file("../WI/WI.shp")
WIcatch = pd.read_pickle("WICatch.df")
lc = pd.read_pickle("Landcover/land_cover.df")

hu = gpd.GeoDataFrame.from_file("DaneCountyData/HydrologicUnits.shp")
intdrain = gpd.GeoDataFrame.from_file("DaneCountyData/InternallyDrained.shp")

In [9]:
huc10s = [709000205, 709000206]

YRivers = WIRivers[WIRivers.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)
YLakes  = WILakes[WILakes.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)
Yhuc8   = WIhuc8[WIhuc8.huc8.isin([7090002])].copy(deep = True).reset_index(drop = True)
Yhuc10  = WIhuc10[WIhuc10.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)
Yhuc12  = WIhuc12[WIhuc12.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)
Ycatch  = WIcatch[WIcatch.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)
Ylc = lc[lc.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)

In [10]:
Ylc_3071 = Ylc.to_crs("EPSG:3071")
Ylc["area_sqkm"] = Ylc_3071.area / 1e6

In [11]:
YRivers.to_pickle("../yahara_data/YRivers.df")
YLakes.to_pickle("../yahara_data/YLakes.df")
Yhuc8.to_pickle("../yahara_data/YHUC8.df")
Yhuc10.to_pickle("../yahara_data/YHUC10.df")
Yhuc12.to_pickle("../yahara_data/YHUC12.df")
Ycatch.to_pickle("../yahara_data/Ycatch.df")
Ylc.to_pickle("../yahara_data/Ylc.df")

In [12]:
hu["huc12"] = hu.HUC12.values.astype(float)

hu["huc10"] = np.floor(hu.huc12.values / 100)

huc10s = [709000205, 709000206]

Yhu = hu[hu.huc10.isin(huc10s)]

Yhu = Yhu.to_crs("EPSG:4326")

Yhu["catchment"] = 0

Yhu_catch = pd.DataFrame(columns = Yhu.columns)

for i in tqdm(range(len(Ycatch))):
    mask = Ycatch.geometry.iloc[i]
    
    catchment_val = Ycatch.GRIDCODE.iloc[i]
    
    clipping = gpd.clip(Yhu, mask)
    clipping["catchment"] = catchment_val
    
    Yhu_catch = Yhu_catch.append(clipping)
    
Yhu_gdf = gpd.GeoDataFrame(Yhu_catch, crs = "EPSG:4326").reset_index(drop = True)

Yhu_gdf_3071 = Yhu_gdf.to_crs("EPSG:3071")

Yhu_gdf["area_sqkm"] = Yhu_gdf_3071.area / 1e6

intdrain = intdrain.to_crs("EPSG:4326")

intdrain["huc12"] = intdrain.HUC12.values.astype(float)

intdrain["huc10"] = np.floor(intdrain.huc12.values / 100).astype(int)

Yintdrain = intdrain[intdrain.huc10.isin(huc10s)].copy(deep = True).reset_index(drop = True)

Yintdrain["catchment"] = 0

Yintdrain_catch = pd.DataFrame(columns = Yintdrain.columns)

for i in tqdm(range(len(Ycatch))):
    mask = Ycatch.geometry.iloc[i]
    
    catchment_val = Ycatch.GRIDCODE.iloc[i]
    
    clipping = gpd.clip(Yintdrain, mask)
    clipping["catchment"] = catchment_val
    
    Yintdrain_catch = Yintdrain_catch.append(clipping)
    
Yintdrain_gdf = gpd.GeoDataFrame(Yintdrain_catch, crs = "EPSG:4326").reset_index(drop = True)    


Yintdrain_gdf_3071 = Yintdrain_gdf.to_crs("EPSG:3071")

Yintdrain_gdf["area_sqkm"] = Yintdrain_gdf_3071.area / 1e6

100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [00:47<00:00,  3.46it/s]


In [13]:
Yhu_gdf.to_pickle("../yahara_data/Yhu.df")
Yintdrain_gdf.to_pickle("../yahara_data/Yintdrain.df")